In [6]:
import sys

from fase import HEAAN
import fase.HEAAN as he
import numpy as np
import os 
import time
import pickle

FN_KEYS = ["ENCRYPTION.txt",
           "MULTIPLICATION.txt",
           "ROTATION_1.txt"]

class Param():
    def __init__(self, n=None, logn=None, logp=None, logq=None, logQboot=None):
        self.n = n
        self.logn = logn
        self.logp = logp
        self.logq = logq 
        self.logQboot = logQboot
        if self.logn == None:
            self.logn = int(np.log2(n))

def key_found(key_path):
    all_found = []
    for fn in FN_KEYS:
        this_fn = key_path + fn
        found = os.path.isfile(this_fn)
        all_found.append(found)
        print(f"{this_fn} is","found" if found else "missing" )
    
    return np.all(all_found)

def decrypt(scheme, secretKey, enc, parms):
    featurized = scheme.decrypt(secretKey, enc)
    arr = np.zeros(parms.n, dtype=np.complex128)
    featurized.__getarr__(arr)
    return arr.real

def encrypt(scheme, val, parms):
    print("[ENCRYPT] 1")
    ctxt = HEAAN.Ciphertext()#logp, logq, n)
    print("[ENCRYPT] 2")
    vv = np.zeros(parms.n) # Need to initialize to zero or will cause "unbound"
    print("[ENCRYPT] 3")
    vv[:len(val)] = val
    print("[ENCRYPT] 4")
    print(vv[:len(val)])
    scheme.encrypt(ctxt, HEAAN.Double(vv), 
                   parms.n, parms.logp, parms.logq)
    print("[ENCRYPT] 5")
    del vv
    return ctxt


In [7]:
class HEAAN_Encryptor():
#    def __init__(self, e_key, lock, key_path):
    def __init__(self, key_path):
        #lock.acquire()# 이렇게 하는건가? 

        logq = 540
        logp = 30
        logn = 14
        n = 1*2**logn

        self.parms = Param(n=n, logp=logp, logq=logq)
        self.key_path = key_path
        print("[ENCRYPTOR] key path", key_path)

        do_reduction = False
        is_serialized = True

        ring = HEAAN.Ring()
        
        #e_key.wait()
        #if not key_found(key_path):
        #    self.get_keys()

        self.scheme = HEAAN.Scheme(ring, True, key_path)
        self.algo = HEAAN.SchemeAlgo(self.scheme)
        self.scheme.loadLeftRotKey(1)
        print("[Encryptor] HEAAN is ready")

        # Check HEAAN
        val = np.arange(10)
        ctxt = encrypt(self.scheme, val, self.parms)
        print(ctxt.n, ctxt.logp, ctxt.logq)
        del ctxt
        #lock.release()

    def _quick_check(self):
        scheme = self.scheme

        return True

    def get_keys(self):
        #print("good to go") 
        #sk = q1.get()
        pass

    def start_encrypt_loop(self, q1, e_sk, e_enc):
        scheme = self.scheme
    
        i=0
        while True:
            e_sk.wait()
            print("[Encryptor] good to go") 
            sk = q1.get()
            e_enc.set()
            print("[Encryptor] e_enc set?", e_enc.is_set())
            if not 'skeleton' in sk.keys():
                raise LookupError("Can't find skeleton in queue")    
            print("[Encryptor] Got a skeleton, Encrypting...")
            print("[Encryptor] ", sk["skeleton"])
            fn = f"/home/hoseung/Work/Kinect_BBS_demo/ctx_{i}.dat"
            #ctx1 = encrypt(scheme, sk['skeleton'], self.parms)



            val = np.arange(10)
            print(self.parms.n, self.parms.logp, self.parms.logq)
            ctx1 = encrypt(scheme, val, self.parms)
            print(ctx1.n, ctx1.logp, ctx1.logq)
            print("[Encryptor] Ctxt encrypted")
            he.SerializationUtils.writeCiphertext(ctx1, fn)
            print("[Encryptor] Ctxt wrote")

            q1.put({"fn_enc_skeleton": fn})
            print("[Encryptor] skeleton encrypted and saved as", fn)
            e_enc.set() # Tell encryption is done and file is ready
            del ctx1 

            time.sleep(1)
            print("Waiting...")


In [8]:
def run_encryptor(q1, e_sk, e_enc, key_path="./"):
    key_path = '/home/hoseung/Work/Kinect_BBS_demo/'
    henc = HEAAN_Encryptor(key_path)
    #print(henc.prams.n)
    henc.start_encrypt_loop(q1, e_sk, e_enc)


def run_temp_qt(q1, e_sk, e_enc):
    #fn = "/home/hoseung/Work/Kinect_BBS_demo/G1/000/BT/bodytracking_data.pickle"
    #skeleton = pickle.load(open(fn, "rb"))
    skeleton = np.arange(10)
    time.sleep(5)
    q1.put({"skeleton":skeleton})
    print("Loaded and put skeleton")
    e_sk.set()
    
    e_enc.wait()
    ctxt = q1.get()
    print(ctxt)
    sys.exit()
    

In [9]:
import multiprocessing 
from multiprocessing import Process, Queue

e_enc = multiprocessing.Event()
e_enc.clear()

In [10]:
q1 = Queue()
e_sk = multiprocessing.Event()
e_sk.clear()
e_enc = multiprocessing.Event()

#lock = multiprocessing.Lock()

fn = "/home/hoseung/Work/Kinect_BBS_demo/G1/000/BT/bodytracking_data.pickle"
skeleton = pickle.load(open(fn, "rb"))

In [11]:
p_enc = Process(target=run_encryptor, args=(q1, e_sk, e_enc), daemon=False)
p_enc.start()

p_qt = Process(target=run_temp_qt, args=(q1, e_sk, e_enc), daemon=False)
p_qt.start()

[ENCRYPTOR] key path /home/hoseung/Work/Kinect_BBS_demo/
[Encryptor] HEAAN is ready
[ENCRYPT] 1
[ENCRYPT] 2
[ENCRYPT] 3
[ENCRYPT] 4
[0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
loading Keys from /home/hoseung/Work/Kinect_BBS_demo/serkey/ENCRYPTION.txt
loading Keys from /home/hoseung/Work/Kinect_BBS_demo/serkey/MULTIPLICATION.txt
loading Keys from /home/hoseung/Work/Kinect_BBS_demo/serkey/ROTATION_1.txt
[ENCRYPT] 5
16384 30 540
Loaded and put skeleton
[Encryptor] good to go
[Encryptor] e_enc set? True
[Encryptor] Got a skeleton, Encrypting...
[Encryptor]  [0 1 2 3 4 5 6 7 8 9]
16384 30 540
[ENCRYPT] 1
[ENCRYPT] 2
[ENCRYPT] 3
[ENCRYPT] 4
[0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
